# Find Planned Observations for JWST Using astroquery
<p><i>By Susan Mullally (smullally@stsci.edu)</i></p>
Below we go through a few examples for how to use the MAST API to find observations planned for JWST (the GTO and ERS programs).  We are going to do this by using [astroquery](https://astroquery.readthedocs.io/en/latest/mast/mast.html) on the MAST CAOM database. To avoid unintentional duplications, JWST proposers are required to check their proposed observations against those already approved.  These tools may be useful in that process, and to find out more about what counts as a duplicate observations, see the [JWST documentation](https://jwst-docs.stsci.edu/display/JPP/JWST+Duplication+Checking).

We begin by writing a couple of functions to query the MAST. The first function does a cone search on the database of JWST observations that have calibration level equal to -1 (proposed observations). You need only provide the RA, Dec and radius of your search.  The second function does a name look-up using the Mast.Name.Lookup service to determine RA and Dec.


In [ ]:
import astropy
from astroquery.mast import Mast

def filteredConeSearch(ra,dec,radius,service="",myfilters=None, returnNum=False):
    """
    This function performs a cone search on the MAST CAOM database
    and returns whether any observations overlap with the cone search
    and other filters provided. This only searches planned observations.
    
    Args:
    ra: right ascension in degrees
    dec: declination in degrees
    radius: radius of cone search in arc seconds
    myfilters: Dictionary of what you want to filter on, 
                if None, it searhes mission=JWST and calib_level=-1
    service:  For testing you can change the MAST service to the testbed here.
    returnNum: False. Set to True if you only want the number of observations returned.
    
    Returns:
    results dictionary unless there are more than 1000 observations or less than 1.
    
    """
    
    
    if service=="":
        service="Mast.Caom.Filtered.Position"
    
    if myfilters!=None:
        filters=myfilters
    else:
        filters = [
                {"paramName":"calib_level",
                          "values":["-1"],},
                {"paramName":"obs_collection","values":["JWST"]}
              ]
    cone_search="%f, %f, %f" % (ra,dec,radius/3600)
    
    #First see how many observations there are using COUNT_BIG(*)
    params =  { "columns":"COUNT_BIG(*)",
                "filters":filters,
                "position":cone_search
          }
    
    result=Mast.service_request(service,params)
    numbObs=int(result[0][0])
    if (numbObs > 1000) | (numbObs == 0) | (returnNum):
        return result
    else:
        print("Found: %u" % numbObs)
    
    params =  { "columns":"*",
            "filters":filters,
            "position":cone_search
              }
    
    result=Mast.service_request(service,params)
    
    return result


def getMASTCoords(name):
    """
    Use Mast.Name.Lookup to get the ra/dec of your target.
    """
    service = 'Mast.Name.Lookup'
    params ={'input':name,
             'format':'json'}
    response = Mast.service_request_async(service,params)
    result = response[0].json()
    coord=result['resolvedCoordinate']
    ra = coord[0]['ra']
    dec = coord[0]['decl']
    
    return(ra,dec)

## Example 1. Look-up TRAPPIST-1 and return all JWST observations on that target.

In [ ]:
target_name="Trappist-1"

(ra,dec) = getMASTCoords(target_name)
print("RA: %f, Dec: %f" % (ra,dec))

### Find planned JWST observations with the function we wrote above. 
Print out the columns that may be of interest.  Notice, it is still up to you to determine if these observations count as a duplicate with those you were planning. For instance, it does not provide the timing information necessary to determine which TRAPPIST-1 planet they are targetting. In some cases, the target_name or proposal title (obs_title) contains this information.

In [ ]:
service="Mast.Caom.Filtered.Position"
radius = 10 #in arc seconds.
result=filteredConeSearch(ra,dec,radius,service=service)
print(result['target_name','proposal_pi','instrument_name','filters','t_exptime','proposal_id'])
print("\nObservation Titles")
print(result['obs_title'])

### Example 1 cont. --  Find HST observations for Trappist-1  using the same function.
We need to build our own filters dictionary to send into the function. 
In this case we want HST observations taken with the WFC3/IR instrument and the F139M filter. Note, limiting your search by the database "filters" column can be tricky for JWST because there is frequently more than one filter listed in the filters field. 

In [ ]:
filters = [{"paramName":"obs_collection",
            "values":["HST"]},
           { "paramName":"instrument_name",
           "values":["WFC3/IR"]},
          {"paramName" : "filters",
          "values" : ["F139M"]}]

service="Mast.Caom.Filtered.Position"

result=filteredConeSearch(ra,dec,radius,service=service,myfilters=filters)

print(result['target_name','proposal_pi','instrument_name','filters','t_exptime'])


## Example 2. Search for JWST planned observations around a list of stars with known planetary disks.
We will only ask for data taken with NIRCam. And we will ask our function to only return the number of observations.

In [ ]:
pdlist = ["Vega", "Deneb","Fomalhaut","HL Tauri","Eta Corvi","HD 15115"]

filters= [ {"paramName": "calib_level",
           "values" :["-1"],
          "paramName": "instrument_name",
           "values":["NIRCam"]}    
]

service="Mast.Caom.Filtered.Position"

radius = 100  #arcseconds for cone search
numlist=[]
for pd in pdlist:
    (ra,dec) = getMASTCoords(pd)
    result=filteredConeSearch(ra,dec,radius,service=service,myfilters=filters,returnNum=True)
    numbObs=int(result[0][0])
    numlist.append(numbObs)

print(pdlist)
print(numlist)

### Check Details
Now I list the details of the observations for Fomalhaut.

In [ ]:
(ra,dec) = getMASTCoords('Fomalhaut')
result = filteredConeSearch(ra,dec,radius,service=service,myfilters=filters)
print(result['target_name','proposal_pi','instrument_name','filters','t_exptime'])